<a href="https://colab.research.google.com/github/flinkflora17/ML_kurzus/blob/main/hazi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from tensorflow import keras
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from google.colab import files
import io
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [9]:
uploaded = files.upload()

Saving weather_data_1hr.csv to weather_data_1hr.csv


In [54]:
weather = pd.read_csv(io.BytesIO(uploaded['weather_data_1hr.csv']))

In [55]:
weather.head()

,loc_id,date,time,isdaytime,tempC,tempF,windspeedMiles,windspeedKmph,winddirdegree,winddir16point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibilityKm,visibilityMiles,pressureMB,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF
0,1,2019-06-01,0,no,16,61,6,10,228,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,77,10,6,1023,31,7,16,61,12,54,16,61,7,12,16,61
1,1,2019-06-01,100,no,16,60,6,9,230,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,78,10,6,1023,31,6,16,60,12,53,16,60,7,11,16,60
2,1,2019-06-01,200,no,15,59,5,9,233,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,79,10,6,1022,31,5,15,59,11,53,15,59,6,10,15,59
3,1,2019-06-01,300,no,15,58,5,8,235,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,79,10,6,1022,31,4,15,58,11,52,14,58,6,9,14,58
4,1,2019-06-01,400,no,15,59,4,7,229,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,77,10,6,1022,31,4,15,59,11,52,15,59,5,8,15,59


In [56]:
weather = weather.drop(labels = ['loc_id', 'time', 'tempF', 'windspeedMiles'], axis = 1)

In [18]:
weather.head()

,date,isdaytime,tempC,windspeedKmph,winddirdegree,winddir16point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibilityKm,visibilityMiles,pressureMB,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF
0,2019-06-01,no,16,10,228,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,77,10,6,1023,31,7,16,61,12,54,16,61,7,12,16,61
1,2019-06-01,no,16,9,230,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,78,10,6,1023,31,6,16,60,12,53,16,60,7,11,16,60
2,2019-06-01,no,15,9,233,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,79,10,6,1022,31,5,15,59,11,53,15,59,6,10,15,59
3,2019-06-01,no,15,8,235,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,79,10,6,1022,31,4,15,58,11,52,14,58,6,9,14,58
4,2019-06-01,no,15,7,229,SW,116,https://cdn.worldweatheronline.net/images/wsym...,Partly cloudy,0.0,0.0,77,10,6,1022,31,4,15,59,11,52,15,59,5,8,15,59


In [57]:
weather = weather.drop(labels = ['winddir16point', 'weatherIconUrl', 'precipInches', 'visibilityMiles', 'pressureInches',
                                 'HeatIndexF', 'DewPointF', 'WindChillF', 'WindGustMiles', 'FeelsLikeF'], axis = 1)

In [20]:
weather.isna().sum()

date             0
isdaytime        0
tempC            0
windspeedKmph    0
winddirdegree    0
weatherCode      0
weatherDesc      0
precipMM         0
humidity         0
visibilityKm     0
pressureMB       0
cloudcover       0
HeatIndexC       0
DewPointC        0
WindChillC       0
WindGustKmph     0
FeelsLikeC       0
dtype: int64

In [21]:
weather.head()

,date,isdaytime,tempC,windspeedKmph,winddirdegree,weatherCode,weatherDesc,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC
0,2019-06-01,no,16,10,228,116,Partly cloudy,0.0,77,10,1023,7,16,12,16,12,16
1,2019-06-01,no,16,9,230,116,Partly cloudy,0.0,78,10,1023,6,16,12,16,11,16
2,2019-06-01,no,15,9,233,116,Partly cloudy,0.0,79,10,1022,5,15,11,15,10,15
3,2019-06-01,no,15,8,235,116,Partly cloudy,0.0,79,10,1022,4,15,11,14,9,14
4,2019-06-01,no,15,7,229,116,Partly cloudy,0.0,77,10,1022,4,15,11,15,8,15


In [28]:
from pandas import unique

In [31]:
weather.weatherDesc.unique()

array(['Partly cloudy', 'Cloudy', 'Overcast', 'Light drizzle',
       'Light rain shower', 'Patchy rain possible', 'Moderate rain',
       'Moderate rain at times', 'Light rain', 'Heavy rain',
       'Patchy light rain', 'Moderate or heavy rain shower',
       'Moderate or heavy rain with thunder', 'Torrential rain shower',
       'Clear', 'Sunny'], dtype=object)

In [58]:
laberer = LabelEncoder()
weather.weatherDesc = laberer.fit_transform(weather.weatherDesc)

In [34]:
weather.head()

,date,isdaytime,tempC,windspeedKmph,winddirdegree,weatherCode,weatherDesc,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC
0,2019-06-01,no,16,10,228,116,11,0.0,77,10,1023,7,16,12,16,12,16
1,2019-06-01,no,16,9,230,116,11,0.0,78,10,1023,6,16,12,16,11,16
2,2019-06-01,no,15,9,233,116,11,0.0,79,10,1022,5,15,11,15,10,15
3,2019-06-01,no,15,8,235,116,11,0.0,79,10,1022,4,15,11,14,9,14
4,2019-06-01,no,15,7,229,116,11,0.0,77,10,1022,4,15,11,15,8,15


In [36]:
weather.weatherDesc.unique()

array([11,  1, 10,  3,  5, 13,  8,  9,  4,  2, 12,  6,  7, 15,  0, 14])

In [42]:
description = {
    '0' : 'Partly Cloudy',
    '1' : 'Felhos',
    '2' : 'Overcast',
    '3' : 'Light Drizzle',
    '4' : 'Light rain shower',
    '5' : 'Patchy rain possible',
    '6' : 'Moderate rain',
    '7' : 'Moderate rain at times',
    '8' : 'Light rain',
    '9' : 'Heavy rain',
    '10' : 'Patchy light rain',
    '11' : 'Moderate or heavy rain shower',
    '12' : 'Moderate or heavy rain with thunder',
    '13' : 'Torrential rain shower',
    '14' : 'Clear',
    '15' : 'Sunny'
}

In [59]:
y = weather.weatherDesc
x = weather.drop('weatherDesc', axis = 1)

In [60]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, shuffle = True)

In [39]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1317, 16) (1317,) (147, 16) (147,)


In [ ]:
#adatok átformázása

#hiányzik

In [61]:
input = Input(shape = (x_train.shape[1]))
x = Dense(32, activation = 'relu')(input)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
output = Dense(1, activation = 'softmax')(x)

In [62]:
model = keras.Model(input, output)

In [63]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
early_stopping_callback = EarlyStopping(patience=10, restore_best_weights=True)

In [65]:
model.fit(x_train, y_train, batch_size=50, epochs=200, validation_split=0.1, callbacks=[early_stopping_callback])

ValueError: ignored

In [53]:
model.predict_on_batch(x_test)

ValueError: ignored

In [ ]:
model.evaluate(x_test, y_test)